Pretraining the IMDb model for finetuning on the compound word prediction task

In [1]:
from __future__ import division, print_function, absolute_import

import tflearn
import numpy as np
import pandas as pd
import random
import json
import tensorflow.compat.v1 as tf
from tflearn.data_utils import to_categorical, pad_sequences
from tflearn.datasets import imdb
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.embedding_ops import embedding
from tflearn.layers.recurrent import bidirectional_rnn, BasicLSTMCell
from tflearn.layers.estimator import regression


/Users/kalliehuynh/miniforge3/envs/stenv/lib/python3.8/site-packages/jax/_src/lib/__init__.py:34: UserWarning: JAX on Mac ARM machines is experimental and minimally tested. Please see https://github.com/google/jax/issues/5501 in the event of problems.
  warnings.warn("JAX on Mac ARM machines is experimental and minimally tested. "


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
np.random.seed(1)
random.seed(1)
tf.set_random_seed(1)

In [3]:
test_neg = pd.read_csv('test_neg.csv')


In [4]:
test_pos = pd.read_csv('test_pos.csv')
train_neg = pd.read_csv('train_neg.csv')
train_pos = pd.read_csv('train_pos.csv')

In [5]:
train = [train_neg, train_pos]
train = pd.concat(train)
train = train.drop('Unnamed: 0', axis=1)

In [6]:
test = [test_neg, test_pos]
test = pd.concat(test)
test = test.drop('Unnamed: 0', axis=1)

In [7]:
test = test.sample(frac=1).reset_index(drop=True)
train = train.sample(frac=1).reset_index(drop=True)

In [8]:
trainX = [[np.array(json.loads(embedding), dtype='float32') for embedding in train.iloc[i, :100]] for i in range(350)]


In [10]:
trainY = [int(i) for i in train.iloc[:350, -1]]

In [11]:
testX = [[np.array(json.loads(embedding), dtype='float32') for embedding in test.iloc[i, :100]] for i in range(175)]

                                                 word_00  \
0      [[-0.167062942089733, 0.1052116373842762, 0.17...   
1      [[0.1040415732190503, 0.1000910442357138, 0.24...   
2      [[-0.167062942089733, 0.1052116373842762, 0.17...   
3      [[0.2583571057403827, 0.2545965502061464, 0.11...   
4      [[-0.3279381770666981, 0.1269941372943537, 0.1...   
...                                                  ...   
24995  [[0.0597157799774428, 0.2024493176656686, 0.19...   
24996  [[-0.167062942089733, 0.1052116373842762, 0.17...   
24997  [[0.2583571057403827, 0.2545965502061464, 0.11...   
24998  [[0.2583571057403827, 0.2545965502061464, 0.11...   
24999  [[-0.167062942089733, 0.1052116373842762, 0.17...   

                                                 word_01  \
0      [[0.4452949018238277, 0.381905943230363, 0.472...   
1      [[-0.1099969348851542, -0.0353920206883006, 0....   
2      [[0.4749093903272134, 0.3018275609945992, 0.17...   
3      [[0.1430436907671176, 0.35106408

In [12]:
testY = [int(i) for i in test.iloc[:175, -1]]

In [12]:
trainX = np.array(trainX)
trainX = trainX.reshape(350, 100, 768)
trainY = np.array(trainY)
trainY = trainY.reshape(350, 1)
testX = np.array(testX)
testX = testX.reshape(175, 100, 768)
testY = np.array(testY)
testY = testY.reshape(175, 1)


NameError: name 'trainX' is not defined

In [ ]:
trainY = to_categorical(trainY)
testY = to_categorical(testY)

In [ ]:
trainY = trainY.reshape((350, 2))
testY = testY.reshape((175, 2))


In [ ]:
tf.reset_default_graph()

In [ ]:
net = input_data(shape=[None, 100, 768])
net = tflearn.lstm(net, 128, dropout=0.8)
net = tflearn.fully_connected(net, 2, activation='softmax')
net = tflearn.regression(net, optimizer='adam', learning_rate=0.0005,
                         loss='categorical_crossentropy')

Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
model = tflearn.DNN(net, tensorboard_verbose=0)


2022-08-09 22:32:22.600855: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [ ]:
model.fit(trainX, trainY, validation_set=(testX, testY), show_metric=True, n_epoch=101, batch_size=4)

Training Step: 5191  | total loss: 0.02992 | time: 9.518s
| Adam | epoch: 101 | loss: 0.02992 - acc: 0.9919 -- iter: 348/350
Training Step: 5192  | total loss: 0.02705 | time: 10.699s
| Adam | epoch: 101 | loss: 0.02705 - acc: 0.9927 | val_loss: 2.74971 - val_acc: 0.5600 -- iter: 350/350
--
